<a href="https://colab.research.google.com/github/ekacs/simadu_analytical_insight/blob/main/analisa_kinerja_TL_pengaduan_(01012019_07062022).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Persiapan
bahan analisa dari dbsimadu

Export dbsimadu to csv, saving on My Drive/Pokja SIMADU 2022/Simadu jango 2022

syntax querinya:

SELECT t_pengaduan.kode, t_pengaduan.ref_username, t_status_history.ref_username, t_status_history.pic, t_pengaduan.ref_admin, t_pengaduan.insert_datetime, t_pengaduan.update_user, t_status_history.update_datetime, tr_status.detil, tr_jenis_aduan.detil, tr_unit.detil FROM t_pengaduan LEFT JOIN t_status_history ON t_pengaduan.id = t_status_history.ref_id LEFT JOIN tr_status ON t_pengaduan.ref_status = tr_status.id LEFT JOIN tr_jenis_aduan ON t_pengaduan.ref_jenis_aduan_admin = tr_jenis_aduan.id LEFT JOIN tr_unit ON t_pengaduan.ref_unit_terlapor_admin = tr_unit.id WHERE insert_datetime>2018 AND ref_admin<>'NULL' ORDER BY insert_datetime ASC;

https://drive.google.com/file/d/1JzWv4O13ZbJTlELwo9QwJN5SuQoan0ia/view?usp=sharing
filename: t_pengaduan.csv


# koding awal

In [3]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import gzip
#pre-start
from google.colab import drive
drive.mount('/content/drive')
datas = pd.read_csv('/content/drive/My Drive/Pokja SIMADU 2022/Simadu jango 2022/t_pengaduan.csv') #nama_file sesuaikan dengan nama file yang diupload
#datas.describe()
#datas.info()

datas.columns = ['kd_track', 'kd_pengadu', 'verifikator', 'auto info', 'korwil', 'insert_datetime', 'auditor_korwil', 'update_datetime', 'status_pengaduan', 'jenis_pengaduan', 'unit_eselon1']
#datas.head(10)
from datetime import timedelta
datas['update_datetime'] = pd.to_datetime(datas['update_datetime'])
datas['insert_datetime'] = pd.to_datetime(datas['insert_datetime'])
datas['lama_TL']=(datas['update_datetime']-datas['insert_datetime'])
#datas.info()
datas2 = datas[(datas.verifikator != 'admin')]
datas2.sort_values(by=['lama_TL'], ascending=True)
datas3 = datas2.drop_duplicates(subset=['kd_track','kd_pengadu'], keep='last')
from datetime import timedelta
datas['update_datetime'] = pd.to_datetime(datas['update_datetime'])
datas['insert_datetime'] = pd.to_datetime(datas['insert_datetime'])
datas['lama_TL']=(datas['update_datetime']-datas['insert_datetime'])
datas3.info()

#analisa1
#status pengaduan per korwil dr th 2019 sd terakhir

#sstatus pengaduan per korwil yang masuk tahun 2019
korwil_2019 = datas3[(datas3['insert_datetime'] >= "2019-01-01") & (datas3['insert_datetime']< "2020-01-01")]
korwil_2019a = korwil_2019.groupby(['korwil','status_pengaduan']).agg({'status_pengaduan':'count'})
#sstatus pengaduan per korwil yang masuk tahun 2020
korwil_2020 = datas3[(datas3['insert_datetime'] >= "2020-01-01") & (datas3['insert_datetime']< "2021-01-01")]
korwil_2020a = korwil_2020.groupby(['korwil','status_pengaduan']).agg({'status_pengaduan':'count'})
#sstatus pengaduan per korwil yang masuk tahun 2021
korwil_2021 = datas3[(datas3['insert_datetime'] >= "2021-01-01") & (datas3['insert_datetime']< "2022-01-01")]
korwil_2021a = korwil_2021.groupby(['korwil','status_pengaduan']).agg({'status_pengaduan':'count'})
#sstatus pengaduan per korwil yang masuk tahun 2022
korwil_2022 = datas3[(datas3['insert_datetime'] >= "2022-01-01") & (datas3['insert_datetime']< "2023-01-01")]
korwil_2022a = korwil_2022.groupby(['korwil','status_pengaduan']).agg({'status_pengaduan':'count'})
#analisa2
#performa korwil thd pengaduan masuk di 2019
korwil_2019b = korwil_2019[(korwil_2019.status_pengaduan != 'Baru') & (korwil_2019.status_pengaduan != 'Proses')]
korwil_2019b = korwil_2019b.groupby(['korwil','status_pengaduan']).agg({'lama_TL':'mean'})
#Rata2 performa korwil thd pengaduan masuk di 2020
korwil_2020b = korwil_2020[(korwil_2020.status_pengaduan != 'Baru') & (korwil_2020.status_pengaduan != 'Proses')]
korwil_2020b = korwil_2020b.groupby(['korwil','status_pengaduan']).agg({'lama_TL':'mean'})
#Rata2 performa korwil thd pengaduan masuk di 2021
korwil_2021b = korwil_2021[(korwil_2021.status_pengaduan != 'Baru') & (korwil_2021.status_pengaduan != 'Proses')]
korwil_2021b = korwil_2021b.groupby(['korwil','status_pengaduan']).agg({'lama_TL':'mean'})
#Rata2 performa korwil thd pengaduan masuk di 2022
korwil_2022b = korwil_2022[(korwil_2022.status_pengaduan != 'Baru') & (korwil_2022.status_pengaduan != 'Proses')]
korwil_2022b = korwil_2022b.groupby(['korwil','status_pengaduan']).agg({'lama_TL':'mean'})

Mounted at /content/drive
<class 'pandas.core.frame.DataFrame'>
Int64Index: 410 entries, 1 to 1764
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype          
---  ------            --------------  -----          
 0   kd_track          410 non-null    object         
 1   kd_pengadu        410 non-null    object         
 2   verifikator       410 non-null    object         
 3   auto info         367 non-null    object         
 4   korwil            410 non-null    object         
 5   insert_datetime   410 non-null    datetime64[ns] 
 6   auditor_korwil    409 non-null    object         
 7   update_datetime   410 non-null    datetime64[ns] 
 8   status_pengaduan  410 non-null    object         
 9   jenis_pengaduan   410 non-null    object         
 10  unit_eselon1      410 non-null    object         
 11  lama_TL           410 non-null    timedelta64[ns]
dtypes: datetime64[ns](2), object(9), timedelta64[ns](1)
memory usage: 41.6+ KB


#data tabel terperincinya
(awal 2019 sd terakhir)
sumber analisa

In [4]:
datas3

,kd_track,kd_pengadu,verifikator,auto info,korwil,insert_datetime,auditor_korwil,update_datetime,status_pengaduan,jenis_pengaduan,unit_eselon1,lama_TL
1,qpacN,fal1509,198212032006041001,Tidak terdapat uraian pengaduan serta data dukung,korwil2,2019-06-03 23:47:56,198212032006041001,2019-07-12 09:08:15,Ditolak,-- Pilih Jenis Pelanggaran --,-- Pilih Direktorat --,38 days 09:20:19
4,xSndY,fendiya,198212032006041001,File Bukti telah diupload untuk melengkapi Pel...,korwil2,2019-06-06 06:30:43,198212032006041001,2020-07-13 08:00:24,Selesai,-- Pilih Jenis Pelanggaran --,-- Pilih Direktorat --,403 days 01:29:41
6,IFsJL,galih,198212032006041001,198701082009122006 telah menjadi verifikator u...,korwil3,2019-06-30 22:16:15,198212032006041001,2020-08-19 18:57:59,Baru,-- Pilih Jenis Pelanggaran --,-- Pilih Direktorat --,415 days 20:41:44
8,TuL3e,ferry67,198212032006041001,NaN,korwil2,2019-07-09 00:48:30,198212032006041001,2019-08-12 10:05:37,Proses,-- Pilih Jenis Pelanggaran --,-- Pilih Direktorat --,34 days 09:17:07
10,4O4UI,mahrusali85,198212032006041001,NaN,korwil2,2019-07-09 13:40:56,198212032006041001,2019-08-12 10:05:04,Proses,-- Pilih Jenis Pelanggaran --,-- Pilih Direktorat --,33 days 20:24:08
...,...,...,...,...,...,...,...,...,...,...,...,...
1754,XM8sd,susiair,19850212,korwil2 telah menjadi verifikator untuk aduan ...,korwil4,2022-06-27 08:25:42,korwil4,2022-06-30 09:51:41,Dilimpahkan,-- Pilih Jenis Pelanggaran --,-- Pilih Direktorat --,3 days 01:25:59
1757,5cjoj,taqwatoallah,korwil5,NaN,korwil5,2022-06-30 15:42:33,korwil5,2022-07-01 10:04:16,Proses,Pelayanan Publik,Direktorat Jenderal Perkeretaapian,0 days 18:21:43
1759,j8M6u,sonyluhkum,19850212,korwil2 telah menjadi verifikator untuk aduan ...,korwil2,2022-07-09 12:13:46,NaN,2022-07-11 19:25:53,Baru,-- Pilih Jenis Pelanggaran --,-- Pilih Direktorat --,2 days 07:12:07
1762,UYvZA,operator2,korwil3,NaN,korwil3,2022-07-12 09:17:29,korwil3,2022-07-15 10:13:35,Proses,Pengaduan Barang dan Jasa,Badan Pengembangan Sumber Daya Manusia Perhubu...,3 days 00:56:06


# analisa 1
statistik data status pengaduan yang masuk

In [5]:
korwil_2019a

status_pengaduan
korwil  status_pengaduan                  
korwil1 Dilimpahkan                      2
        Proses                           3
korwil2 Ditolak                          7
        Proses                           5
        Selesai                          5
korwil3 Baru                            11
        Proses                           2
        Selesai                          2
korwil4 Proses                           5
        Selesai                          1
korwil5 Proses                           1

In [6]:
korwil_2020a

status_pengaduan
korwil  status_pengaduan                  
korwil1 Ditolak                          7
        Proses                           1
        Selesai                         17
korwil2 Ditolak                          1
        Selesai                         33
korwil3 Proses                           1
        Selesai                         48
korwil4 Ditolak                          2
        Selesai                         25
korwil5 Ditolak                          2
        Proses                           1
        Selesai                         28

In [7]:
korwil_2021a

status_pengaduan
korwil  status_pengaduan                  
korwil1 Ditolak                          1
        Selesai                         12
korwil2 Dilimpahkan                      1
        Ditolak                          4
        Proses                           1
        Selesai                         24
korwil3 Dilimpahkan                      4
        Ditolak                          2
        Proses                           7
        Selesai                         14
korwil4 Dilimpahkan                      1
        Ditolak                          7
        Proses                           1
        Selesai                         26
korwil5 Dilimpahkan                      3
        Ditolak                          2
        Proses                           4
        Selesai                          7

In [8]:
korwil_2022a

status_pengaduan
korwil  status_pengaduan                  
korwil1 Dilimpahkan                      2
        Proses                           2
        Selesai                         10
korwil2 Baru                             1
        Dilimpahkan                      1
        Ditolak                          1
        Proses                           8
        Selesai                          4
korwil3 Dilimpahkan                      5
        Ditolak                          1
        Proses                           8
        Selesai                          2
korwil4 Dilimpahkan                      6
        Ditolak                          6
        Proses                           5
        Selesai                          5
korwil5 Baru                             1
        Dilimpahkan                      2
        Ditolak                          1
        Proses                           3
        Selesai                          5

# analisa 2
rata2 performa tiap korwil menindaklanjuti pengaduan per status 

In [9]:
korwil_2019b

lama_TL
korwil  status_pengaduan                           
korwil1 Dilimpahkan        315 days 23:03:29.500000
korwil2 Ditolak          16 days 22:04:53.428571428
        Selesai            184 days 13:06:27.200000
korwil3 Selesai                    30 days 06:44:53
korwil4 Selesai                    82 days 01:49:18

In [10]:
korwil_2020b

lama_TL
korwil  status_pengaduan                            
korwil1 Ditolak           69 days 02:39:11.857142857
        Selesai                    117 days 12:20:07
korwil2 Ditolak                     64 days 17:50:08
        Selesai          181 days 13:34:05.636363636
korwil3 Selesai             125 days 11:35:16.562500
korwil4 Ditolak                     18 days 18:20:37
        Selesai             178 days 02:19:29.320000
korwil5 Ditolak             180 days 08:50:10.500000
        Selesai          230 days 11:30:26.071428572

In [11]:
korwil_2021b

lama_TL
korwil  status_pengaduan                            
korwil1 Ditolak                      0 days 01:22:38
        Selesai           69 days 11:52:15.666666667
korwil2 Dilimpahkan                 28 days 19:26:09
        Ditolak               8 days 09:23:25.500000
        Selesai              95 days 20:11:00.125000
korwil3 Dilimpahkan                  9 days 16:40:06
        Ditolak               0 days 19:27:58.500000
        Selesai           88 days 02:21:57.428571429
korwil4 Dilimpahkan                  0 days 11:20:51
        Ditolak            4 days 12:00:42.285714285
        Selesai           79 days 16:41:09.923076923
korwil5 Dilimpahkan       12 days 19:49:51.666666666
        Ditolak                     28 days 21:40:02
        Selesai          123 days 06:13:33.285714286

In [12]:
korwil_2022b

lama_TL
korwil  status_pengaduan                          
korwil1 Dilimpahkan               13 days 12:28:47
        Selesai            78 days 13:15:23.100000
korwil2 Dilimpahkan                0 days 00:03:13
        Ditolak                    8 days 21:42:30
        Selesai            40 days 10:36:53.500000
korwil3 Dilimpahkan         8 days 04:12:54.200000
        Ditolak                    4 days 10:24:23
        Selesai                  112 days 19:24:29
korwil4 Dilimpahkan                4 days 11:05:01
        Ditolak          1 days 20:52:32.166666666
        Selesai                   49 days 06:38:04
korwil5 Dilimpahkan        15 days 20:39:05.500000
        Ditolak                   19 days 02:04:07
        Selesai            47 days 09:53:39.800000

# export to CSV


In [ ]:
'''
drive.mount('/content/drive', force_remount=True)
path = '/content/drive/My Drive/Pokja SIMADU 2022/Simadu jango 2022/'
datas3.to_csv(path+'analisa kinerja TL pengaduan (01012019-07062022).csv', index=None)
'''


# export to google-sheet
bakalan jadi bahan olah google-studio


In [ ]:
#key = 1xJGScFPlQb7faPApmzWBnEV7PSIvGiVC_rkfGpAjJ94
#baca deh https://docs.gspread.org/en/latest/index.html


In [13]:
#!pip install --upgrade gspread
#!pip install gspread-dataframe
#!pip install --upgrade --quiet gspread

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

# Go to https://sheets.google.com to see your new spreadsheet.


from gspread_dataframe import get_as_dataframe, set_with_dataframe

sh = gc.create('analisa_kinerja_TL_pengaduan')
#sh = gc.create(brandname + 'analisa_kinerja_TL_pengaduan_rev1')
worksheet = gc.open('analisa_kinerja_TL_pengaduan_rev1').sheet1

set_with_dataframe(worksheet, datas3)


# google data studio
https://datastudio.google.com/reporting/fcdeb54c-9760-434f-8d17-0467aaf92662/page/p_5ld7kyjuwc